In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessi

/kaggle/input/train-dataset/train.csv
/kaggle/input/detbert3weights/MLTC_model_statedetbert.bin69
/kaggle/input/test-dataset/test.csv
/kaggle/input/roberta3weights/MLTC_model_stateroberta3.bin5
/kaggle/input/sample-submission/sample_submission.csv
/kaggle/input/detbet6weights/MLTC_model_statedet6.bin69
/kaggle/input/debertalargeweights/MLTC_model_statedet6.bin75
/kaggle/input/roberta6weights/MLTC_model_stateroberta6.bin5


In [2]:
MAX_LEN = 256
TEST_BATCH_SIZE = 4

In [3]:
from transformers import DebertaTokenizer, DebertaModel

In [4]:
from transformers import RobertaTokenizer,RobertaModel 

In [5]:
tokenizer1 = DebertaTokenizer.from_pretrained('microsoft/deberta-base', truncation=True, do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

In [6]:
tokenizer2 = DebertaTokenizer.from_pretrained('microsoft/deberta-large',truncation=True, do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

In [7]:
tokenizer4 = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [8]:
tokenizer3 = DebertaTokenizer.from_pretrained('microsoft/deberta-base', truncation=True, do_lower_case=True)

In [9]:
import torch

In [10]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.tokenizer = tokenizer
        self.df = df
        self.title = list(df['combined'])
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'title': title
        }


In [18]:

target_list =['Categories',
 'cs.AI',
 'cs.AR',
 'cs.CE',
 'cs.CL',
 'cs.CR',
 'cs.CV',
 'cs.DB',
 'cs.DC',
 'cs.DM',
 'cs.GT',
 'cs.IR',
 'cs.IT',
 'cs.LG',
 'cs.LO',
 'cs.NI',
 'cs.OS',
 'cs.PL',
 'cs.RO',
 'cs.SD',
 'cs.SE',
 'econ.EM',
 'econ.GN',
 'econ.TH',
 'eess.AS',
 'eess.IV',
 'eess.SP',
 'math.AC',
 'math.AP',
 'math.AT',
 'math.CO',
 'math.CV',
 'math.GR',
 'math.IT',
 'math.LO',
 'math.NT',
 'math.PR',
 'math.QA',
 'math.ST',
 'q-bio.BM',
 'q-bio.CB',
 'q-bio.GN',
 'q-bio.MN',
 'q-bio.NC',
 'q-bio.TO',
 'q-fin.CP',
 'q-fin.EC',
 'q-fin.GN',
 'q-fin.MF',
 'q-fin.PM',
 'q-fin.PR',
 'q-fin.RM',
 'q-fin.TR',
 'stat.AP',
 'stat.CO',
 'stat.ME',
 'stat.ML',
 'stat.TH',
 'combined']

In [19]:
target_list = target_list[1:-1]
print(target_list)

['cs.AI', 'cs.AR', 'cs.CE', 'cs.CL', 'cs.CR', 'cs.CV', 'cs.DB', 'cs.DC', 'cs.DM', 'cs.GT', 'cs.IR', 'cs.IT', 'cs.LG', 'cs.LO', 'cs.NI', 'cs.OS', 'cs.PL', 'cs.RO', 'cs.SD', 'cs.SE', 'econ.EM', 'econ.GN', 'econ.TH', 'eess.AS', 'eess.IV', 'eess.SP', 'math.AC', 'math.AP', 'math.AT', 'math.CO', 'math.CV', 'math.GR', 'math.IT', 'math.LO', 'math.NT', 'math.PR', 'math.QA', 'math.ST', 'q-bio.BM', 'q-bio.CB', 'q-bio.GN', 'q-bio.MN', 'q-bio.NC', 'q-bio.TO', 'q-fin.CP', 'q-fin.EC', 'q-fin.GN', 'q-fin.MF', 'q-fin.PM', 'q-fin.PR', 'q-fin.RM', 'q-fin.TR', 'stat.AP', 'stat.CO', 'stat.ME', 'stat.ML', 'stat.TH']


In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [21]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


In [22]:
class  DebertaClass(torch.nn.Module):
    def __init__(self):
        super(DebertaClass, self).__init__()
        self.bert_model = DebertaModel.from_pretrained('microsoft/deberta-base',return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 57)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        cls_representation = output.last_hidden_state[:, 0]
        output_dropout = self.dropout(cls_representation)
        output = self.linear(output_dropout)
        return output



debertamodel = DebertaClass()
debertamodel.to(device)



DebertaClass(
  (bert_model): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermediate): De

In [23]:
class  DebertaLargeClass(torch.nn.Module):
    def __init__(self):
        super(DebertaLargeClass, self).__init__()
        self.bert_model = DebertaModel.from_pretrained("microsoft/deberta-large",return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(1024, 57)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        cls_representation = output.last_hidden_state[:, 0]
        output_dropout = self.dropout(cls_representation)
        output = self.linear(output_dropout)
        return output


deberta_largemodel = DebertaLargeClass()
deberta_largemodel.to(device)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

DebertaLargeClass(
  (bert_model): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=1024, out_features=3072, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=1024, out_features=1024, bias=False)
              (pos_q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (inte

In [24]:
class  RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.bert_model = RobertaModel.from_pretrained("roberta-base",return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 57)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output


robertamodel = RobertaClass()
robertamodel.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (bert_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

In [25]:
from transformers import AdamW
from sklearn.metrics import f1_score
optimizer = AdamW(debertamodel.parameters(), lr = 1e-5) 
optimizer1 = AdamW(deberta_largemodel.parameters(), lr = 1e-5) 
optimizer2 = AdamW(debertamodel.parameters(), lr = 1e-5) 
optimizer3 = AdamW(robertamodel.parameters(), lr = 1e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


yaha load kar lena 

In [26]:
debertmodel = DebertaClass()
debertamodel.load_state_dict(torch.load("/kaggle/input/detbert3weights/MLTC_model_statedetbert.bin69"))
debertamodel = debertamodel.to(device)

In [27]:
debertamodel1 = DebertaClass()
debertamodel1.load_state_dict(torch.load("/kaggle/input/detbet6weights/MLTC_model_statedet6.bin69"))
debertamodel1 = debertamodel1.to(device)

In [28]:
deberta_largemodel = DebertaLargeClass()
deberta_largemodel.load_state_dict(torch.load("/kaggle/input/debertalargeweights/MLTC_model_statedet6.bin75"))
deberta_largemodel = deberta_largemodel.to(device)

In [29]:
robertamodel = RobertaClass()
robertamodel.load_state_dict(torch.load('/kaggle/input/roberta3weights/MLTC_model_stateroberta3.bin5'))
robertamodel.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (bert_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

In [30]:
def treatingzero_cases(predictions,prediction_probs):
    all_zeros_mask = torch.all(predictions == 0, dim=1)
    rows_with_all_zeros = torch.nonzero(all_zeros_mask).squeeze()
    for idx in rows_with_all_zeros:
        idx_value=torch.argmax(prediction_probs[idx,:])
        predictions[idx,idx_value]=1
    return predictions

In [31]:
class testDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = list(df['combined'])
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'title': title
        }

In [32]:
tt=pd.read_csv('/kaggle/input/test-dataset/test.csv')

In [33]:
tt['combined']=tt['Title']+tt['Abstract']

In [34]:
df_dataset = testDataset(tt, tokenizer1, MAX_LEN)
df_dataset1 = testDataset(tt, tokenizer2, MAX_LEN)
df_dataset2 = testDataset(tt, tokenizer3, MAX_LEN)
df_dataset3 = testDataset(tt, tokenizer4, MAX_LEN)

In [35]:
tt_data_loader = torch.utils.data.DataLoader(df_dataset, 
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
tt_data_loader1 = torch.utils.data.DataLoader(df_dataset1, 
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
tt_data_loader2 = torch.utils.data.DataLoader(df_dataset2, 
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
tt_data_loader3 = torch.utils.data.DataLoader(df_dataset3, 
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [36]:
def get_predictions_test(model, model1, model2, model3, data_loader, data_loader1, data_loader2, data_loader3):
    model = model.eval()
    
    titles = []
    predictions = []
    prediction_probs = []

    with torch.no_grad():
        for data, data1, data2, data3 in zip(data_loader, data_loader1, data_loader2, data_loader3):
            title = data["title"]
            ids = data["input_ids"].to(device, dtype=torch.long)
            mask = data["attention_mask"].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
          
            outputs = model(ids, mask, token_type_ids)
            

            ids1 = data1["input_ids"].to(device, dtype=torch.long)
            mask1 = data1["attention_mask"].to(device, dtype=torch.long)
            token_type_ids1 = data1['token_type_ids'].to(device, dtype=torch.long)
           
            outputs1 = model1(ids1, mask1, token_type_ids1)
            
            ids2 = data2["input_ids"].to(device, dtype=torch.long)
            mask2 = data2["attention_mask"].to(device, dtype=torch.long)
            token_type_ids2 = data2['token_type_ids'].to(device, dtype=torch.long)
          
            outputs2 = model2(ids2, mask2, token_type_ids2)
            
            ids3 = data3["input_ids"].to(device, dtype=torch.long)
            mask3 = data3["attention_mask"].to(device, dtype=torch.long)
            token_type_ids3 = data3['token_type_ids'].to(device, dtype=torch.long)
          
            outputs3 = model3(ids3, mask3, token_type_ids3)

            combined_outputs = (outputs + outputs1 + outputs2 + outputs3)/4
            combined_outputs = torch.sigmoid(combined_outputs).detach().cpu()
            custom_threshold = 0.47
            preds = (combined_outputs >= custom_threshold).float()

          

            titles.extend(title)
            predictions.extend(preds)
            prediction_probs.extend(combined_outputs)
          

    predictions = torch.stack(predictions)
    prediction_probs = torch.stack(prediction_probs)

    return predictions, prediction_probs


In [37]:
 predictions2,prediction_probs2 = get_predictions_test(debertamodel, deberta_largemodel, debertamodel1, robertamodel, tt_data_loader, tt_data_loader1, tt_data_loader2, tt_data_loader3)

In [38]:
predictions2=treatingzero_cases(predictions2,prediction_probs2)

In [39]:
def add_predictions_to_dataframe(tt, predictions, class_names):
    for i, class_name in enumerate(class_names):
        tt[class_name] = [pred[i].item() for pred in predictions]

    return tt

In [40]:
tt_df = add_predictions_to_dataframe(tt, predictions2,target_list)

In [41]:
rows_without_1 = ~tt_df.eq(1).any(axis=1)

In [42]:
num_rows_without_1 = rows_without_1.sum()

In [43]:
print("Number of rows without 1 in any column:", num_rows_without_1)

Number of rows without 1 in any column: 0


In [44]:
tt_df=tt_df.drop(columns=['Title','Abstract','combined'],axis=1)

In [45]:
tt_df

,Id,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,30332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,50337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,66515,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,57464,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,43169,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,41708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10970,38843,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10971,57571,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10972,31964,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
sample = pd.read_csv('/kaggle/input/sample-submission/sample_submission.csv')

In [47]:
tt_df = tt_df.reindex(columns = sample.columns)

In [48]:
tt_df

,Id,math.AT,stat.AP,cs.AR,math.QA,q-bio.MN,eess.AS,eess.IV,stat.ME,econ.GN,...,cs.SE,cs.RO,stat.TH,cs.DB,math.ST,q-bio.GN,q-fin.PM,q-bio.TO,math.GR,cs.IR
0,30332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,50337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,66515,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,57464,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,43169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,41708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10970,38843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10971,57571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10972,31964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
tt_df.to_csv('Kameng.csv', index=False)

In [50]:
tt_df

,Id,math.AT,stat.AP,cs.AR,math.QA,q-bio.MN,eess.AS,eess.IV,stat.ME,econ.GN,...,cs.SE,cs.RO,stat.TH,cs.DB,math.ST,q-bio.GN,q-fin.PM,q-bio.TO,math.GR,cs.IR
0,30332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,50337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,66515,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,57464,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,43169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,41708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10970,38843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10971,57571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10972,31964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
